In [8]:
from collections import deque
import pickle
import random

def bfs(tree, start):
    visited = set([start])
    queue = deque([(start, 0)])
    distances = {}
    
    while queue:
        node, distance = queue.popleft()
        distances[node] = distance
        
        for neighbor in tree[node]:
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append((neighbor, distance + 1))
    
    return distances

def calculate_m_values(tree, k):
    m_values = []
    for node in range(len(tree)):
        distances = bfs(tree, node)
        for h in range(len(tree)):
            if len([x for x in distances.values() if x <= h]) >= k:
                m_values.append(h)
                break
    
    return m_values

def greedy_labeling(tree, k, m_values):
    nodes = sorted(range(len(tree)), key=lambda x: m_values[x])
    labels = [None] * len(tree)
    
    step = 0
    while True:
        for node in nodes:
            if labels[node] is None:
                available_labels = set(range(k))
                distances = bfs(tree, node)
                neighbors = [x for x in distances if distances[x] <= m_values[node]]
                neighbors_p = neighbors[1:]
                random.shuffle(neighbors_p)
                neighbors[1:] = neighbors_p
                
                for neighbor in neighbors:
                    if labels[neighbor] in available_labels:
                        available_labels.remove(labels[neighbor])
                    if not available_labels:
                        break
                
                available_labels = list(available_labels)
                random.shuffle(available_labels)
                
                if available_labels:
                    for neighbor in neighbors:
                        if labels[neighbor] is None:
                            labels[neighbor] = available_labels.pop()
                        if not available_labels:
                            break
                        
                unique_labels = set()
                if available_labels:
                    for neighbor in neighbors:
                        if labels[neighbor] in unique_labels:
                            labels[neighbor] = available_labels.pop()
                            if not available_labels:
                                break
                        unique_labels.add(labels[neighbor])
            else:
                missing_labels = set(range(k))
                distances = bfs(tree, node)
                neighbors = [x for x in distances if distances[x] <= m_values[node]]
                neighbors_p = neighbors[1:]
                random.shuffle(neighbors_p)
                neighbors[1:] = neighbors_p
                
                for neighbor in neighbors:
                    if labels[neighbor] in missing_labels:
                        missing_labels.remove(labels[neighbor])
                    if not missing_labels:
                        break
                
                missing_labels = list(missing_labels)
                random.shuffle(missing_labels)
                
                if not missing_labels:
                    continue
                else:
                    for neighbor in neighbors:
                        if labels[neighbor] is None:
                            labels[neighbor] = missing_labels.pop()
                        if not missing_labels:
                            break
                        
                    unique_labels = set()
                    if missing_labels:
                        for neighbor in neighbors:
                            if labels[neighbor] in unique_labels:
                                labels[neighbor] = missing_labels.pop()
                                if not missing_labels:
                                    break
                            unique_labels.add(labels[neighbor])
            
        count = 0
        for node in nodes:
            missing_labels = set(range(k))
            distances = bfs(tree, node)
            neighbors = [x for x in distances if distances[x] <= m_values[node]+step]

            for neighbor in neighbors:
                if labels[neighbor] in missing_labels:
                    missing_labels.remove(labels[neighbor])
                if not missing_labels:
                    break

            if not missing_labels:
                count += 1
            
        if count == len(tree):
            break
        else:
            step += 0.001

    for i in range(len(labels)):
        if labels[i] is None:
            labels[i] = 0
    
    return labels

def load_input(file_adjlist, file_k_values):
    with open(file_adjlist, 'rb') as f:
        adj_list_of_trees = pickle.load(f)
    with open(file_k_values, 'rb') as f:
        k_values = pickle.load(f)
    
    return zip(adj_list_of_trees, k_values)    

def save_output(file_name, output_list):
    with open(file_name, 'wb') as f:
        pickle.dump(output_list, f)

def main():
    # Load input instances
    # data_input = load_input('Examples_of_AdjLists_of_Trees', 'Examples_of_k_values')
    # data_input = load_input('Small_Examples_of_AdjLists_of_Trees', 'Small_Examples_of_k_values')
    # data_input = load_input('Medium_Examples_of_AdjLists_of_Trees', 'Medium_Examples_of_k_values')
    data_input = load_input('Large_Examples_of_AdjLists_of_Trees', 'Large_Examples_of_k_values')

    # Run algorithm and save results
    
    results = []
    index = 0
    for tree, k in data_input:
        m_values = calculate_m_values(tree, k)
        labels = greedy_labeling(tree, k, m_values)
        results.append(labels)
        index += 1
        print(index)
    
    #check result
    print(results)

    # save_output('example_solutions', results)
    # save_output('small_solutions', results)
    # save_output('medium_solutions', results)
    save_output('large_solutions', results)

if __name__ == '__main__':
    main()

1
2
3
4
5


KeyboardInterrupt: 